In [374]:
# import required libraries
import pandas as pd
import numpy as np
import os
import json
import re

## Data Loading
I loaded in the files for each search location, extracted the dictionaries and turned them into columns, and concatenated them all to build one dataset per search location. In the end, I have three dataframes. One for DC, one for US, and one with both. I also added a column for the search location to know which data came from where.

This code for the DC search.

In [375]:
# path to the folder containing data files
folder_path = '../Data/DC_Search/'

# create an empty list to store the dataframes
dfs = []

# loop over all files in the folder
for filename in os.listdir(folder_path):
    # check if the file is a JSON file
    if filename.endswith('.json'):
        # read JSON file
        with open(os.path.join(folder_path, filename)) as f:
            data = json.load(f)

        # extract the "jobs_results" dictionary
        jobs_results = data['jobs_results']

        # convert the dictionary into a DataFrame
        df = pd.DataFrame.from_dict(jobs_results)

        # append the dataframe to the list of dataframes
        dfs.append(df)

# concatenate all the dataframes into one
final_df = pd.concat(dfs, ignore_index=True)

### extracting dictionaries from detected_extensions column
# Assuming your DataFrame is named "df"
def extract_dict_columns(d):
    """Custom function to extract dictionary keys into separate columns"""
    keys = ["schedule_type", "salary", "posted_at", "work_from_home"]
    return {
        "schedule_type": d.get("schedule_type", np.nan),
        "salary": d.get("salary", np.nan),
        "posted_at": d.get("posted_at", np.nan),
        "work_from_home": d.get("work_from_home", np.nan)
    }

# Use apply method with the custom function to extract the dictionary keys into separate columns
final_df[["schedule_type", "salary", "posted_at", "work_from_home"]] = final_df["detected_extensions"].apply(lambda x: pd.Series(extract_dict_columns(x)))

# Drop the original column containing the nested dictionaries
final_df = final_df.drop(["detected_extensions", "job_id", "related_links", "extensions"], axis=1)

# get qualifications, responsibilities, and benefits
quals = pd.json_normalize(pd.json_normalize(final_df['job_highlights'])[0])['items']
resps = pd.json_normalize(pd.json_normalize(final_df['job_highlights'])[1])['items']
bens = pd.json_normalize(pd.json_normalize(final_df['job_highlights'])[2])['items']

# add to dataframe as columns
final_df['Qualifications'] = quals
final_df['Responsibilities'] = resps
final_df['Benefits'] = bens

# drop job_highlights column
final_df = final_df.drop(['job_highlights'], axis=1)

# add a column for the search location
final_df['Search_Location'] = 'DC'

# rename it to DC_search
DC_search = final_df

Repeat the same process for the US search.

In [376]:
# path to the folder containing data files
folder_path = '../Data/USA_Search/'

# create an empty list to store the dataframes
dfs = []

# loop over all files in the folder
for filename in os.listdir(folder_path):
    # check if the file is a JSON file
    if filename.endswith('.json'):
        # read JSON file
        with open(os.path.join(folder_path, filename)) as f:
            data = json.load(f)

        # extract the "jobs_results" dictionary
        jobs_results = data['jobs_results']

        # convert the dictionary into a DataFrame
        df = pd.DataFrame.from_dict(jobs_results)

        # append the dataframe to the list of dataframes
        dfs.append(df)

# concatenate all the dataframes into one
final_df = pd.concat(dfs, ignore_index=True)

### extracting dictionaries from detected_extensions column
# Assuming your DataFrame is named "df"
def extract_dict_columns(d):
    """Custom function to extract dictionary keys into separate columns"""
    keys = ["schedule_type", "salary", "posted_at", "work_from_home"]
    return {
        "schedule_type": d.get("schedule_type", np.nan),
        "salary": d.get("salary", np.nan),
        "posted_at": d.get("posted_at", np.nan),
        "work_from_home": d.get("work_from_home", np.nan)
    }

# Use apply method with the custom function to extract the dictionary keys into separate columns
final_df[["schedule_type", "salary", "posted_at", "work_from_home"]] = final_df["detected_extensions"].apply(lambda x: pd.Series(extract_dict_columns(x)))

# Drop the original column containing the nested dictionaries
final_df = final_df.drop(["detected_extensions", "job_id", "related_links", "extensions"], axis=1)

# get qualifications, responsibilities, and benefits
quals = pd.json_normalize(pd.json_normalize(final_df['job_highlights'])[0])['items']
resps = pd.json_normalize(pd.json_normalize(final_df['job_highlights'])[1])['items']
bens = pd.json_normalize(pd.json_normalize(final_df['job_highlights'])[2])['items']

# add to dataframe as columns
final_df['Qualifications'] = quals
final_df['Responsibilities'] = resps
final_df['Benefits'] = bens

# drop job_highlights column
final_df = final_df.drop(['job_highlights'], axis=1)

# add a column for the search location
final_df['Search_Location'] = 'USA'

# rename it to DC_search
USA_search = final_df

Concatenate the two dataframes.

In [377]:
# concatenate the two dataframes
all = pd.concat([DC_search, USA_search], ignore_index=True)

## Data Processing and Cleaning

- Removing duplicates
- Strip leading and trailing whitespace from posted_at column
- Clean posted_at column
- Cleaning via column

In [378]:
# remove duplicate rows
all = all.drop_duplicates(subset=['description'], keep='first')

# remove "ago" from posted_at column
all['posted_at'] = all['posted_at'].str.replace('ago', '')

# strip leading or trailing whitespace from posted_at column
all['posted_at'] = all['posted_at'].str.strip()

# change NaNs to not available
all['posted_at'] = all['posted_at'].fillna('Not Available')

# change anything that has hours to 1 day
all['posted_at'] = all['posted_at'].apply(lambda x: '1 day' if 'hour' in x else x)

# remove the one row that has month in posted_at column
all = all[all['posted_at'] != '1 month']

# change "not available" to NaN
all['posted_at'] = all['posted_at'].replace('Not Available', np.nan)

# remove days and day from posted_at column and keep only the number
all['posted_at'] = all['posted_at'].str.replace(' days', '')
all['posted_at'] = all['posted_at'].str.replace(' day', '')

# change posted_at column name to days_posted
all = all.rename(columns={'posted_at': 'days_posted'})

# removing "via" from via column
all['via'] = all['via'].str.replace('via ', '')

# remove trailing and leading whitespace from via column
all['via'] = all['via'].str.strip()

# remove leading and trailing whitespace from all text columns
all['title'] = all['title'].str.strip()
all['company_name'] = all['company_name'].str.strip()
all['location'] = all['location'].str.strip()
all['description'] = all['description'].str.strip()
all['schedule_type'] = all['schedule_type'].str.strip()
all['salary'] = all['salary'].str.strip()

# add salary ranges columns
# change benefit NaNs to a list with "Not Available"
all['Benefits'] = all['Benefits'].fillna('Not Available')
salaries = []

for x in all['Benefits']:
    if x == 'Not Available':
        salaries.append([np.nan])
    elif re.findall(r'\$\d+(?:,\d+)*(?:\.\d+)?', ''.join(x)):
        salaries.append(re.findall(r'\$\d+(?:,\d+)*(?:\.\d+)?', ''.join(x)))
    else:
        salary_found = False
        for y in x:
            if re.findall(r'\$\d+(?:,\d+)*(?:\.\d+)?', y) == []:
                continue
            else:
                salaries.append(re.findall(r'\$\d+(?:,\d+)*(?:\.\d+)?', y))
                salary_found = True
        if not salary_found:
            salaries.append([np.nan])

# return the first and last value of each list within salaries
min_salary = [x[0] for x in salaries]
max_salary = [x[-1] for x in salaries]

# add min and max salary columns to dataframe
all['min_salary'] = min_salary
all['max_salary'] = max_salary

Based on an average work hours per year of 2,087, I calculated the yearly salary for each job posting.

In [379]:
# remove unnecessary text from salary column
sals = all['salary'].str.split(' ', 1).str[0]

# replace NaNs with 0-0
sals = sals.fillna('0-0')

# split the salary column into min and max salary columns
myl = []
for value in sals:
    split_values = re.split("-|–", value)
    myl.append(split_values)

# split myl into two lists
min_sal = [x[0] for x in myl]
max_sal = [x[-1] for x in myl]

# add min and max salary columns to dataframe
all['min_sal'] = min_sal
all['max_sal'] = max_sal

# if min_sal or max_sal has a K, replace it with 000
all['min_sal'] = all['min_sal'].str.replace('K', '000')
all['max_sal'] = all['max_sal'].str.replace('K', '000')

# remove commas from min_sal and max_sal columns
all['min_sal'] = all['min_sal'].str.replace(',', '')
all['max_sal'] = all['max_sal'].str.replace(',', '')

# change min_sal and max_sal columns to numeric
all['min_sal'] = pd.to_numeric(all['min_sal'])
all['max_sal'] = pd.to_numeric(all['max_sal'])

# if min_sal or max_sal is less than 300, multiply by 2087
all['min_sal'] = all['min_sal'].apply(lambda x: x*2087 if x < 300 else x)
all['max_sal'] = all['max_sal'].apply(lambda x: x*2087 if x < 300 else x)

# if min_salary, max_salary are empty, fill them with min_sal, max_sal
all['min_salary'] = all['min_salary'].fillna(all['min_sal'])
all['max_salary'] = all['max_salary'].fillna(all['max_sal'])

# remove min_sal and max_sal columns
all = all.drop(['min_sal', 'max_sal'], axis=1)

# remove $ and , from min_salary and max_salary columns and turn them into floats
all['min_salary'] = all['min_salary'].apply(lambda x: str(x).replace(',', '').replace('$', '') if isinstance(x, str) else x).astype(float)
all['max_salary'] = all['max_salary'].apply(lambda x: str(x).replace(',', '').replace('$', '') if isinstance(x, str) else x).astype(float)

/var/folders/jc/37t7tc411k5_r_7l10g6cy6r0000gn/T/ipykernel_38773/3256195869.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  sals = all['salary'].str.split(' ', 1).str[0]


Find the rows with errors to manually fix them.

In [380]:
import pandas as pd

# index 15
all.loc[15, 'max_salary'] = 245700

# index 94
all.loc[94, 'max_salary'] = 100000
all.loc[94, 'min_salary'] = 80000

# index 105
all.loc[105, 'max_salary'] = 370000
all.loc[105, 'min_salary'] = 200000

# index 218
all.loc[218, 'max_salary'] = 370000
all.loc[218, 'min_salary'] = 200000

# index 224
all.loc[224, 'max_salary'] = 335000
all.loc[224, 'min_salary'] = 230000

# index 258
all.loc[258, 'max_salary'] = 100000
all.loc[258, 'min_salary'] = 80000

# index 344
all.loc[344, 'max_salary'] = 335000
all.loc[344, 'min_salary'] = 196000

# index 481
all.loc[481, 'max_salary'] = 28*2087
all.loc[481, 'min_salary'] = 20*2087

# index 582
all.loc[582, 'max_salary'] = 100000
all.loc[582, 'min_salary'] = 80000

# index 739
all.loc[739, 'max_salary'] = 250128
all.loc[739, 'min_salary'] = 168070


## Feature Engineering

In [387]:
# remove salary column
all = all.drop(['salary'], axis=1)

In [386]:
# create column for range between min and max salary
all['salary_range'] = all['max_salary'] - all['min_salary']

In [389]:
all['Benefits']

0                                          Not Available
1      [(NYC only) Pursuant to Section 8-102 of title...
2                                          Not Available
3                                          Not Available
4                                          Not Available
                             ...                        
806                                        Not Available
807    [US salary range: $150,000 - $250,000, We offe...
812                                        Not Available
815                                        Not Available
820    [Besides a great work environment, our compreh...
Name: Benefits, Length: 578, dtype: object